In [13]:
import pandas as pd
import numpy as np
import os
from functools import reduce
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.spatial import distance


In [17]:
# --------------------------------------------------
# UPDATE THIS PATH TO WHERE YOUR DATASETS ARE STORED
# Example:
# Windows OneDrive:
# DATA_DIR = r"C:\Users\YourName\OneDrive\crop_yield_datasets"
#
# Mac:
# DATA_DIR = "/Users/YourName/OneDrive/crop_yield_datasets"
# --------------------------------------------------

DATA_DIR = r"C:\Users\Harshada\OneDrive - University of Sussex\MLP Dataset\ML 2025 Coursework Dataset - A1"

In [18]:
def read_data(base_dir):
    Tveg = pd.read_csv(os.path.join(base_dir, "TVeg_tavg_data.csv"))
    TwS = pd.read_csv(os.path.join(base_dir, "TWS_inst_data.csv"))
    yield__and_production = pd.read_csv(os.path.join(base_dir, "Yield_and_Production_data.csv"))
    country_Latitude_longitude = pd.read_csv(
        os.path.join(base_dir, "country_latitude_longitude_area_lookup.csv")
    )

    soiltmp40_100 = pd.read_csv(os.path.join(base_dir, "SoilTMP40_100cm_inst_data.csv"))
    soiltmp10_40 = pd.read_csv(os.path.join(base_dir, "SoilTMP10_40cm_inst_data.csv"))
    soiltemp100_200 = pd.read_csv(os.path.join(base_dir, "SoilTMP100_200cm_inst_data.csv"))
    soiltemp0_10 = pd.read_csv(os.path.join(base_dir, "SoilTMP0_10cm_inst_data.csv"))

    soilmoi100_200 = pd.read_csv(os.path.join(base_dir, "SoilMoi100_200cm_inst_data.csv"))
    soilmoi10_40 = pd.read_csv(os.path.join(base_dir, "SoilMoi10_40cm_inst_data.csv"))
    soilmoi40_100 = pd.read_csv(os.path.join(base_dir, "SoilMoi40_100cm_inst_data.csv"))
    soilmoi0_10 = pd.read_csv(os.path.join(base_dir, "SoilMoi0_10cm_inst_data.csv"))

    snowf_tavg = pd.read_csv(os.path.join(base_dir, "Snowf_tavg_data.csv"))
    rain_tavg = pd.read_csv(os.path.join(base_dir, "Rainf_tavg_data.csv"))
    land_cover = pd.read_csv(os.path.join(base_dir, "Land_cover_percent_data.csv"))
    Esoil_tavg = pd.read_csv(os.path.join(base_dir, "ESoil_tavg_data.csv"))
    canoplnt_tavg = pd.read_csv(os.path.join(base_dir, "CanopInt_inst_data.csv"))

    return (
        Tveg, TwS, yield__and_production, country_Latitude_longitude,
        soiltmp40_100, soiltmp10_40, soiltemp100_200, soiltemp0_10,
        soilmoi100_200, soilmoi10_40, soilmoi40_100, soilmoi0_10,
        snowf_tavg, rain_tavg, land_cover, Esoil_tavg, canoplnt_tavg
    )


In [ ]:
datasets = read_data(DATA_DIR)

dataset_names = [
    "TVeg", "TWS", "Yield and Production", "Country Lookup",
    "SoilTMP40_100", "SoilTMP10_40", "SoilTMP100_200", "SoilTMP0_10",
    "SoilMoi100_200", "SoilMoi10_40", "SoilMoi40_100", "SoilMoi0_10",
    "Snowf_tavg", "Rainf_tavg", "Land Cover", "ESoil_tavg", "CanopInt_tavg"
]

In [ ]:
def summarize_dataframes(dataframes, names):
    """
    Display .head() and .describe() for a list of dataframes.
    """
    for df, name in zip(dataframes, names):
        print(f"\nSummary for: {name}")
        print("Head:")
        print(df.head())
        print("\nDescribe:")
        print(df.describe(include="all"))
        print("-" * 60)

summarize_dataframes(datasets, dataset_names)


In [ ]:
base_dir = "ml_dataset"
datasets = read_data(base_dir)

dataset_names = [
    "TVeg", "TWS", "Yield and Production", "Country Lookup",
    "SoilTMP40_100", "SoilTMP10_40", "SoilTMP100_200", "SoilTMP0_10",
    "SoilMoi100_200", "SoilMoi10_40", "SoilMoi40_100", "SoilMoi0_10",
    "Snowf_tavg", "Rainf_tavg", "Land Cover", "ESoil_tavg", "CanopInt_tavg"
]

summarize_dataframes(datasets, dataset_names)



 Summary for: TVeg
 Head:
   longitude  latitude  TVeg_tavg_month_1  TVeg_tavg_month_2  \
0      -71.5     -54.5           1.332788           1.533512   
1      -70.5     -54.5           1.249232           1.139860   
2      -69.5     -54.5           1.533212           1.052825   
3      -68.5     -54.5           8.139574           4.556105   
4      -67.5     -54.5          15.640950          10.241190   

   TVeg_tavg_month_3  TVeg_tavg_month_4  TVeg_tavg_month_5  TVeg_tavg_month_6  \
0           0.777899           0.042899           0.001958           0.002436   
1           0.767769           0.026896           0.006372           0.006287   
2           0.794003           0.018582           0.006218           0.004723   
3           2.563924           0.179212           0.005834           0.004620   
4           5.407119           0.476426           0.002584           0.003083   

   TVeg_tavg_month_7  TVeg_tavg_month_8  TVeg_tavg_month_9  \
0           0.004742           0.008161

In [ ]:
dataframes_to_process  = {
    'Tveg_tavg': Tveg,
    'TWS_inst': TwS,
    'SoilTMP40_100cm': soiltmp40_100,
    'SoilTMP10_40cm': soiltmp10_40,
    'SoilTMP100_200cm': soiltemp100_200,
    'SoilTMP0_10cm': soiltemp0_10,
    'SoilMoi100_200cm': soilmoi100_200,
    'SoilMoi10_40cm': soilmoi10_40,
    'SoilMoi40_100cm': soilmoi40_100,
    'SoilMoi0_10cm': soilmoi0_10,
    'Snowf_tavg': snowf_tavg,
    'Rainf_tavg': rain_tavg,
    'ESoil_tavg': Esoil_tavg,
    'CanopInt_inst': canoplnt_tavg
}


In [ ]:
def compute_patterned_yearly_average_df(df, var_name, month_prefix="_month_"):
    monthly_cols = [col for col in df.columns if any(col.endswith(f"{month_prefix}{i}") for i in range(1, 13))]

    if not monthly_cols:
        raise ValueError(f"No monthly columns found for {var_name}.")

    df = df.copy()
    df[f'{var_name}_annual_avg'] = df[monthly_cols].mean(axis=1)
    df.drop(columns=monthly_cols, inplace=True)

    return df


In [ ]:
for name, df in dataframes_to_process.items():  # dfs = {name: DataFrame}
    processed_df = compute_patterned_yearly_average_df(df, name)
    print(f"\n=== {name} ===")
    print(processed_df.head())



=== Tveg_tavg ===
   longitude  latitude  year  Tveg_tavg_annual_avg
0      -71.5     -54.5  2010              0.489381
1      -70.5     -54.5  2010              0.478433
2      -69.5     -54.5  2010              0.464609
3      -68.5     -54.5  2010              2.541678
4      -67.5     -54.5  2010              5.152414

=== TWS_inst ===
   longitude  latitude  year  TWS_inst_annual_avg
0      -71.5     -54.5  2010          1410.814167
1      -70.5     -54.5  2010          1425.798500
2      -69.5     -54.5  2010          1240.800583
3      -68.5     -54.5  2010          1304.636750
4      -67.5     -54.5  2010          1289.169417

=== SoilTMP40_100cm ===
   longitude  latitude  year  SoilTMP40_100cm_annual_avg
0      -71.5     -54.5  2010                  278.060492
1      -70.5     -54.5  2010                  276.907442
2      -69.5     -54.5  2010                  275.219875
3      -68.5     -54.5  2010                  276.307492
4      -67.5     -54.5  2010                  2

In [ ]:
yield__and_production.head()

,Domain,Country,Element,Item Code (CPC),Item,Year,Value,Flag
0,Crops and livestock products,Afghanistan,Yield,01341,Apples,2010,7000.0,A
1,Crops and livestock products,Afghanistan,Production,01341,Apples,2010,59850.0,A
2,Crops and livestock products,Afghanistan,Yield,01341,Apples,2011,7000.0,A
3,Crops and livestock products,Afghanistan,Production,01341,Apples,2011,62041.0,A
4,Crops and livestock products,Afghanistan,Yield,01341,Apples,2012,7651.9,A


In [ ]:
# Standardize column names
yield__and_production.columns = yield__and_production.columns.str.strip().str.lower()

# Keep only records where element is 'yield'
yield_df = yield__and_production[yield__and_production['element'].str.lower() == 'yield']

# Reset index
yield_df.reset_index(drop=True, inplace=True)

# Final check
print("Filtered dataset (only Yield) shape:", yield_df.shape)
print(yield_df[['domain','country', 'year', 'item', 'element', 'value']].head())


Filtered dataset (only Yield) shape: (75473, 8)
                         domain      country  year    item element   value
0  Crops and livestock products  Afghanistan  2010  Apples   Yield  7000.0
1  Crops and livestock products  Afghanistan  2011  Apples   Yield  7000.0
2  Crops and livestock products  Afghanistan  2012  Apples   Yield  7651.9
3  Crops and livestock products  Afghanistan  2013  Apples   Yield  7600.5
4  Crops and livestock products  Afghanistan  2014  Apples   Yield  7300.0


In [ ]:
yield_df.head()

,domain,country,element,item code (cpc),item,year,value,flag
0,Crops and livestock products,Afghanistan,Yield,01341,Apples,2010,7000.0,A
1,Crops and livestock products,Afghanistan,Yield,01341,Apples,2011,7000.0,A
2,Crops and livestock products,Afghanistan,Yield,01341,Apples,2012,7651.9,A
3,Crops and livestock products,Afghanistan,Yield,01341,Apples,2013,7600.5,A
4,Crops and livestock products,Afghanistan,Yield,01341,Apples,2014,7300.0,A


In [ ]:
del yield_df['domain']

In [ ]:
yield_df['year'] = yield_df['year'] - 1

C:\Users\Harshada\AppData\Local\Temp\ipykernel_26512\479920114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yield_df['year'] = yield_df['year'] - 1


In [ ]:
yield_df.head()

,country,element,item code (cpc),item,year,value,flag
0,Afghanistan,Yield,01341,Apples,2009,7000.0,A
1,Afghanistan,Yield,01341,Apples,2010,7000.0,A
2,Afghanistan,Yield,01341,Apples,2011,7651.9,A
3,Afghanistan,Yield,01341,Apples,2012,7600.5,A
4,Afghanistan,Yield,01341,Apples,2013,7300.0,A


In [ ]:
country_Latitude_longitude.head()

,centroid latitude,centroid longitude,country,area,centroid radius
0,33.939110,67.709953,Afghanistan,652090,4.555949
1,41.153332,20.168331,Albania,28748,0.956597
2,28.033886,1.659626,Algeria,2381741,8.707076
3,-14.270972,-170.132217,American Samoa,199,0.079589
4,42.546245,1.601554,Andorra,468,0.122053


In [ ]:
land_cover.columns = land_cover.columns.str.strip()
class_cols = [col for col in land_cover.columns if 'class' in col.lower()]

# Step 2: Create a new column with the maximum class percentage per row
land_cover['land_cover_max'] = land_cover[class_cols].max(axis=1)

# Step 3: Drop the original 13 class columns
land_cover.drop(columns=class_cols, inplace=True)

# Step 4: Confirm result
print("Processed land cover shape:", land_cover.shape)
print(land_cover.head())

Processed land cover shape: (644280, 4)
   longitude  latitude  year  land_cover_max
0     -176.5     -59.5  2010             100
1     -175.5     -59.5  2010             100
2     -174.5     -59.5  2010             100
3     -173.5     -59.5  2010             100
4     -172.5     -59.5  2010             100


In [ ]:
land_cover.head()

,longitude,latitude,year,land_cover_max
0,-176.5,-59.5,2010,100
1,-175.5,-59.5,2010,100
2,-174.5,-59.5,2010,100
3,-173.5,-59.5,2010,100
4,-172.5,-59.5,2010,100


In [ ]:
# Correct data directory
data_dir = "ml_dataset"
csv_files = glob.glob(f"{data_dir}/*.csv")

annual_avg_dfs = {}

# Process each file for annual average
for file_path in csv_files:
    df = pd.read_csv(file_path)

    # Match monthly columns (e.g., *_month_1 to *_month_12)
    monthly_cols = [col for col in df.columns if '_month_' in col.lower()]

    if monthly_cols:
        var_name = monthly_cols[0].rsplit('_month_', 1)[0]
        df[f'{var_name}_annual_avg'] = df[monthly_cols].mean(axis=1)
        df_avg = df[['longitude', 'latitude', 'year', f'{var_name}_annual_avg']]
        annual_avg_dfs[var_name] = df_avg

#Merge all processed DataFrames
dfs_to_merge = list(annual_avg_dfs.values())

if dfs_to_merge:
    merged_df = reduce(
        lambda left, right: pd.merge(left, right, on=['longitude', 'latitude', 'year'], how='outer'),
        dfs_to_merge
    )
    print("\nMerged DataFrame shape:", merged_df.shape)
    print(merged_df.head())
else:
    print("No annual average datasets found.")



Merged DataFrame shape: (194298, 17)
   longitude  latitude  year  CanopInt_inst_annual_avg  ESoil_tavg_annual_avg  \
0     -176.5      66.5  2010                  0.021199              17.154338   
1     -176.5      66.5  2011                  0.020344              18.033296   
2     -176.5      66.5  2012                  0.021931              19.889972   
3     -176.5      66.5  2013                  0.022761              17.504125   
4     -176.5      66.5  2014                  0.020658              19.887469   

   Rainf_tavg_annual_avg  Snowf_tavg_annual_avg  \
0               0.000005               0.000010   
1               0.000006               0.000012   
2               0.000006               0.000007   
3               0.000005               0.000010   
4               0.000006               0.000011   

   SoilMoi0_10cm_inst_annual_avg  SoilMoi100_200cm_inst_annual_avg  \
0                      26.407989                        438.930208   
1                      24.78

In [ ]:
# Step 1: Drop duplicates
merged_df_cleaned = merged_df.drop_duplicates()

# Step 2: Drop rows with any NaN/null values
merged_df_cleaned = merged_df_cleaned.dropna()

# Optional: Reset index
merged_df_cleaned.reset_index(drop=True, inplace=True)

# Check final result
print("Cleaned merged DataFrame shape:", merged_df_cleaned.shape)
print(merged_df_cleaned.head())


Cleaned merged DataFrame shape: (194298, 17)
   longitude  latitude  year  CanopInt_inst_annual_avg  ESoil_tavg_annual_avg  \
0     -176.5      66.5  2010                  0.021199              17.154338   
1     -176.5      66.5  2011                  0.020344              18.033296   
2     -176.5      66.5  2012                  0.021931              19.889972   
3     -176.5      66.5  2013                  0.022761              17.504125   
4     -176.5      66.5  2014                  0.020658              19.887469   

   Rainf_tavg_annual_avg  Snowf_tavg_annual_avg  \
0               0.000005               0.000010   
1               0.000006               0.000012   
2               0.000006               0.000007   
3               0.000005               0.000010   
4               0.000006               0.000011   

   SoilMoi0_10cm_inst_annual_avg  SoilMoi100_200cm_inst_annual_avg  \
0                      26.407989                        438.930208   
1                    

In [ ]:
merged_df_cleaned.head()

,longitude,latitude,year,CanopInt_inst_annual_avg,ESoil_tavg_annual_avg,Rainf_tavg_annual_avg,Snowf_tavg_annual_avg,SoilMoi0_10cm_inst_annual_avg,SoilMoi100_200cm_inst_annual_avg,SoilMoi10_40cm_inst_annual_avg,SoilMoi40_100cm_inst_annual_avg,SoilTMP0_10cm_inst_annual_avg,SoilTMP100_200cm_inst_annual_avg,SoilTMP10_40cm_inst_annual_avg,SoilTMP40_100cm_inst_annual_avg,TVeg_tavg_annual_avg,TWS_inst_annual_avg
0,-176.5,66.5,2010,0.021199,17.154338,0.000005,0.000010,26.407989,438.930208,76.372438,152.794492,270.168200,269.068517,270.918725,270.262075,1.865024,1387.448833
1,-176.5,66.5,2011,0.020344,18.033296,0.000006,0.000012,24.785870,438.867867,78.435821,159.316100,269.721758,269.231975,270.624200,270.150625,1.423547,1432.049750
2,-176.5,66.5,2012,0.021931,19.889972,0.000006,0.000007,23.110597,438.982892,73.777794,153.990867,267.992025,268.356092,269.390725,269.069417,2.027180,1385.389417
3,-176.5,66.5,2013,0.022761,17.504125,0.000005,0.000010,23.154087,438.962050,71.322145,149.705025,268.842867,267.859933,269.631342,268.990600,1.765119,1367.282083
4,-176.5,66.5,2014,0.020658,19.887469,0.000006,0.000011,24.021897,438.961500,72.097313,151.713192,270.133958,268.987850,270.668650,270.079450,2.238372,1406.019583


In [ ]:
# Load both datasets (if not already loaded)
# merged_df_cleaned = ...  # already exists
country_lookup = pd.read_csv("ml_dataset/country_latitude_longitude_area_lookup.csv")
country_lookup.columns = country_lookup.columns.str.strip().str.lower()

# Rename centroid columns for easier access
country_lookup.rename(columns={
    'centroid latitude': 'centroid_latitude',
    'centroid longitude': 'centroid_longitude'
}, inplace=True)

# Keep only necessary columns
country_centroids = country_lookup[['country', 'centroid_latitude', 'centroid_longitude']].dropna()

# Step 1: Convert to NumPy arrays for fast computation
points = merged_df_cleaned[['latitude', 'longitude']].to_numpy()
centroids = country_centroids[['centroid_latitude', 'centroid_longitude']].to_numpy()

# Step 2: For each point, compute distance to each country centroid
nearest_country_indices = np.argmin(distance.cdist(points, centroids, metric='euclidean'), axis=1)

# Step 3: Map nearest countries
merged_df_cleaned['country'] = country_centroids.iloc[nearest_country_indices].reset_index(drop=True)['country']

# Step 4: Final result
print("Country-mapped merged_df_cleaned shape:", merged_df_cleaned.shape)
print(merged_df_cleaned[['latitude', 'longitude', 'country']].head())


Country-mapped merged_df_cleaned shape: (194298, 18)
   latitude  longitude   country
0      66.5     -176.5  Kiribati
1      66.5     -176.5  Kiribati
2      66.5     -176.5  Kiribati
3      66.5     -176.5  Kiribati
4      66.5     -176.5  Kiribati


In [ ]:
merged_df_cleaned.head()

,longitude,latitude,year,CanopInt_inst_annual_avg,ESoil_tavg_annual_avg,Rainf_tavg_annual_avg,Snowf_tavg_annual_avg,SoilMoi0_10cm_inst_annual_avg,SoilMoi100_200cm_inst_annual_avg,SoilMoi10_40cm_inst_annual_avg,SoilMoi40_100cm_inst_annual_avg,SoilTMP0_10cm_inst_annual_avg,SoilTMP100_200cm_inst_annual_avg,SoilTMP10_40cm_inst_annual_avg,SoilTMP40_100cm_inst_annual_avg,TVeg_tavg_annual_avg,TWS_inst_annual_avg,country
0,-176.5,66.5,2010,0.021199,17.154338,0.000005,0.000010,26.407989,438.930208,76.372438,152.794492,270.168200,269.068517,270.918725,270.262075,1.865024,1387.448833,Kiribati
1,-176.5,66.5,2011,0.020344,18.033296,0.000006,0.000012,24.785870,438.867867,78.435821,159.316100,269.721758,269.231975,270.624200,270.150625,1.423547,1432.049750,Kiribati
2,-176.5,66.5,2012,0.021931,19.889972,0.000006,0.000007,23.110597,438.982892,73.777794,153.990867,267.992025,268.356092,269.390725,269.069417,2.027180,1385.389417,Kiribati
3,-176.5,66.5,2013,0.022761,17.504125,0.000005,0.000010,23.154087,438.962050,71.322145,149.705025,268.842867,267.859933,269.631342,268.990600,1.765119,1367.282083,Kiribati
4,-176.5,66.5,2014,0.020658,19.887469,0.000006,0.000011,24.021897,438.961500,72.097313,151.713192,270.133958,268.987850,270.668650,270.079450,2.238372,1406.019583,Kiribati


In [ ]:
# Merge land_cover with merged_df_cleaned on ['longitude', 'latitude', 'year']
merged_df_with_land_cover = pd.merge(
    merged_df_cleaned,
    land_cover,
    on=['longitude', 'latitude', 'year'],
    how='left'
)

# Final check
print("Merged shape with land_cover:", merged_df_with_land_cover.shape)
print(merged_df_with_land_cover[['longitude', 'latitude', 'year', 'land_cover_max']].head())  # Adjust if column name differs


Merged shape with land_cover: (194298, 19)
   longitude  latitude  year  land_cover_max
0     -176.5      66.5  2010             100
1     -176.5      66.5  2011             100
2     -176.5      66.5  2012             100
3     -176.5      66.5  2013             100
4     -176.5      66.5  2014             100


In [ ]:
merged_df_with_land_cover.head()

,longitude,latitude,year,CanopInt_inst_annual_avg,ESoil_tavg_annual_avg,Rainf_tavg_annual_avg,Snowf_tavg_annual_avg,SoilMoi0_10cm_inst_annual_avg,SoilMoi100_200cm_inst_annual_avg,SoilMoi10_40cm_inst_annual_avg,SoilMoi40_100cm_inst_annual_avg,SoilTMP0_10cm_inst_annual_avg,SoilTMP100_200cm_inst_annual_avg,SoilTMP10_40cm_inst_annual_avg,SoilTMP40_100cm_inst_annual_avg,TVeg_tavg_annual_avg,TWS_inst_annual_avg,country,land_cover_max
0,-176.5,66.5,2010,0.021199,17.154338,0.000005,0.000010,26.407989,438.930208,76.372438,152.794492,270.168200,269.068517,270.918725,270.262075,1.865024,1387.448833,Kiribati,100
1,-176.5,66.5,2011,0.020344,18.033296,0.000006,0.000012,24.785870,438.867867,78.435821,159.316100,269.721758,269.231975,270.624200,270.150625,1.423547,1432.049750,Kiribati,100
2,-176.5,66.5,2012,0.021931,19.889972,0.000006,0.000007,23.110597,438.982892,73.777794,153.990867,267.992025,268.356092,269.390725,269.069417,2.027180,1385.389417,Kiribati,100
3,-176.5,66.5,2013,0.022761,17.504125,0.000005,0.000010,23.154087,438.962050,71.322145,149.705025,268.842867,267.859933,269.631342,268.990600,1.765119,1367.282083,Kiribati,100
4,-176.5,66.5,2014,0.020658,19.887469,0.000006,0.000011,24.021897,438.961500,72.097313,151.713192,270.133958,268.987850,270.668650,270.079450,2.238372,1406.019583,Kiribati,100


In [ ]:
# Step 1: Drop latitude and longitude (won't be meaningful after aggregation)
df = merged_df_with_land_cover.drop(columns=['latitude', 'longitude'])

# Step 2: Group by 'country' and 'year' and take the mean of all other columns
aggregated_df = df.groupby(['country', 'year'], as_index=False).mean()

# Step 3: Final result
print("Aggregated DataFrame shape:", aggregated_df.shape)
print(aggregated_df.head())


Aggregated DataFrame shape: (2548, 17)
       country  year  CanopInt_inst_annual_avg  ESoil_tavg_annual_avg  \
0  Afghanistan  2010                  0.003434              17.406516   
1  Afghanistan  2011                  0.004174              17.658792   
2  Afghanistan  2012                  0.004708              19.387013   
3  Afghanistan  2013                  0.004051              20.860506   
4  Afghanistan  2014                  0.004531              19.445957   

   Rainf_tavg_annual_avg  Snowf_tavg_annual_avg  \
0               0.000007           9.650432e-07   
1               0.000008           1.733773e-06   
2               0.000007           2.634706e-06   
3               0.000008           1.361736e-06   
4               0.000008           1.729525e-06   

   SoilMoi0_10cm_inst_annual_avg  SoilMoi100_200cm_inst_annual_avg  \
0                      16.522271                        240.593605   
1                      17.474098                        238.593572   
2    

In [ ]:
aggregated_df.head()

,country,year,CanopInt_inst_annual_avg,ESoil_tavg_annual_avg,Rainf_tavg_annual_avg,Snowf_tavg_annual_avg,SoilMoi0_10cm_inst_annual_avg,SoilMoi100_200cm_inst_annual_avg,SoilMoi10_40cm_inst_annual_avg,SoilMoi40_100cm_inst_annual_avg,SoilTMP0_10cm_inst_annual_avg,SoilTMP100_200cm_inst_annual_avg,SoilTMP10_40cm_inst_annual_avg,SoilTMP40_100cm_inst_annual_avg,TVeg_tavg_annual_avg,TWS_inst_annual_avg,land_cover_max
0,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,289.319155,288.908378,289.317581,289.187112,0.665504,688.719626,100.0
1,Afghanistan,2011,0.004174,17.658792,0.000008,1.733773e-06,17.474098,238.593572,67.260410,132.755216,289.398011,288.918274,289.285327,289.169375,0.293760,684.123852,100.0
2,Afghanistan,2012,0.004708,19.387013,0.000007,2.634706e-06,17.885093,239.145373,68.767602,134.561956,288.368113,288.069227,288.356495,288.237801,0.779504,699.515013,100.0
3,Afghanistan,2013,0.004051,20.860506,0.000008,1.361736e-06,18.206896,241.062339,68.528548,135.918621,289.715174,289.097295,289.577175,289.413866,0.850076,699.928255,100.0
4,Afghanistan,2014,0.004531,19.445957,0.000008,1.729525e-06,17.882212,239.098683,68.116610,134.940831,288.845817,288.418973,288.725204,288.591054,0.668922,695.321333,100.0


In [ ]:
# Merge yield_df with aggregated_df on 'country' and 'year'
final_merged_df = pd.merge(aggregated_df, yield_df, on=['country', 'year'], how='inner')

# Final check
print("Final merged DataFrame shape:", final_merged_df.shape)
print(final_merged_df.head())


Final merged DataFrame shape: (62738, 22)
       country  year  CanopInt_inst_annual_avg  ESoil_tavg_annual_avg  \
0  Afghanistan  2010                  0.003434              17.406516   
1  Afghanistan  2010                  0.003434              17.406516   
2  Afghanistan  2010                  0.003434              17.406516   
3  Afghanistan  2010                  0.003434              17.406516   
4  Afghanistan  2010                  0.003434              17.406516   

   Rainf_tavg_annual_avg  Snowf_tavg_annual_avg  \
0               0.000007           9.650432e-07   
1               0.000007           9.650432e-07   
2               0.000007           9.650432e-07   
3               0.000007           9.650432e-07   
4               0.000007           9.650432e-07   

   SoilMoi0_10cm_inst_annual_avg  SoilMoi100_200cm_inst_annual_avg  \
0                      16.522271                        240.593605   
1                      16.522271                        240.593605   
2 

In [ ]:
final_merged_df.head()

,country,year,CanopInt_inst_annual_avg,ESoil_tavg_annual_avg,Rainf_tavg_annual_avg,Snowf_tavg_annual_avg,SoilMoi0_10cm_inst_annual_avg,SoilMoi100_200cm_inst_annual_avg,SoilMoi10_40cm_inst_annual_avg,SoilMoi40_100cm_inst_annual_avg,...,SoilTMP10_40cm_inst_annual_avg,SoilTMP40_100cm_inst_annual_avg,TVeg_tavg_annual_avg,TWS_inst_annual_avg,land_cover_max,element,item code (cpc),item,value,flag
0,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.0,Yield,01341,Apples,7000.0,A
1,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.0,Yield,01343,Apricots,8172.5,E
2,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.0,Yield,0115,Barley,1610.0,A
3,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.0,Yield,01229,Cantaloupes and other melons,10114.5,E
4,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.0,Yield,01315,Figs,4000.0,A


In [ ]:
final_merged_df.isnull().sum()

country                             0
year                                0
CanopInt_inst_annual_avg            0
ESoil_tavg_annual_avg               0
Rainf_tavg_annual_avg               0
Snowf_tavg_annual_avg               0
SoilMoi0_10cm_inst_annual_avg       0
SoilMoi100_200cm_inst_annual_avg    0
SoilMoi10_40cm_inst_annual_avg      0
SoilMoi40_100cm_inst_annual_avg     0
SoilTMP0_10cm_inst_annual_avg       0
SoilTMP100_200cm_inst_annual_avg    0
SoilTMP10_40cm_inst_annual_avg      0
SoilTMP40_100cm_inst_annual_avg     0
TVeg_tavg_annual_avg                0
TWS_inst_annual_avg                 0
land_cover_max                      0
element                             0
item code (cpc)                     0
item                                0
value                               0
flag                                0
dtype: int64

In [ ]:
final_merged_df

,country,year,CanopInt_inst_annual_avg,ESoil_tavg_annual_avg,Rainf_tavg_annual_avg,Snowf_tavg_annual_avg,SoilMoi0_10cm_inst_annual_avg,SoilMoi100_200cm_inst_annual_avg,SoilMoi10_40cm_inst_annual_avg,SoilMoi40_100cm_inst_annual_avg,...,SoilTMP10_40cm_inst_annual_avg,SoilTMP40_100cm_inst_annual_avg,TVeg_tavg_annual_avg,TWS_inst_annual_avg,land_cover_max,element,item code (cpc),item,value,flag
0,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.00000,Yield,01341,Apples,7000.0,A
1,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.00000,Yield,01343,Apricots,8172.5,E
2,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.00000,Yield,0115,Barley,1610.0,A
3,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.00000,Yield,01229,Cantaloupes and other melons,10114.5,E
4,Afghanistan,2010,0.003434,17.406516,0.000007,9.650432e-07,16.522271,240.593605,67.114595,134.640014,...,289.317581,289.187112,0.665504,688.719626,100.00000,Yield,01315,Figs,4000.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62733,Zimbabwe,2021,0.015283,22.604501,0.000013,0.000000e+00,14.270586,239.554960,40.861896,74.049221,...,295.821738,295.642288,7.029131,825.583166,99.72973,Yield,01324,"Tangerines, mandarins, clementines",7669.5,E
62734,Zimbabwe,2021,0.015283,22.604501,0.000013,0.000000e+00,14.270586,239.554960,40.861896,74.049221,...,295.821738,295.642288,7.029131,825.583166,99.72973,Yield,0111,Wheat,5168.0,E
62735,Zimbabwe,2021,0.015283,22.604501,0.000013,0.000000e+00,14.270586,239.554960,40.861896,74.049221,...,295.821738,295.642288,7.029131,825.583166,99.72973,Yield,F1720,"Roots and Tubers, Total",4447.9,E
62736,Zimbabwe,2021,0.015283,22.604501,0.000013,0.000000e+00,14.270586,239.554960,40.861896,74.049221,...,295.821738,295.642288,7.029131,825.583166,99.72973,Yield,F1729,"Treenuts, Total",2183.3,E


In [ ]:
final_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62738 entries, 0 to 62737
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   country                           62738 non-null  object 
 1   year                              62738 non-null  int64  
 2   CanopInt_inst_annual_avg          62738 non-null  float64
 3   ESoil_tavg_annual_avg             62738 non-null  float64
 4   Rainf_tavg_annual_avg             62738 non-null  float64
 5   Snowf_tavg_annual_avg             62738 non-null  float64
 6   SoilMoi0_10cm_inst_annual_avg     62738 non-null  float64
 7   SoilMoi100_200cm_inst_annual_avg  62738 non-null  float64
 8   SoilMoi10_40cm_inst_annual_avg    62738 non-null  float64
 9   SoilMoi40_100cm_inst_annual_avg   62738 non-null  float64
 10  SoilTMP0_10cm_inst_annual_avg     62738 non-null  float64
 11  SoilTMP100_200cm_inst_annual_avg  62738 non-null  float64
 12  Soil

In [ ]:
final_merged_df.drop(['element','item code (cpc)', 'flag'], axis=1, inplace=True)

In [ ]:
# 1. Define columns to drop and identify feature types
excluded_cols = ['year', 'value']
categorical_cols = ['item', 'country']
numerical_cols = [col for col in final_merged_df.columns if col not in excluded_cols + categorical_cols]

# 2. Split dataset by year
train_data = final_merged_df[final_merged_df['year'] < 2021]
val_data = final_merged_df[final_merged_df['year'] == 2021]

# 3. Extract target variables
train_target = train_data['value'].values
val_target = val_data['value'].values

# 4. Drop unused columns
train_data.drop(['value', 'year'], axis=1, inplace=True)
val_data.drop(['value', 'year'], axis=1, inplace=True)

# 5. Categorical transformer: OneHot only
cat_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# 6. Column transformer: keep numeric + one-hot encoded categorical
column_pipeline = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),
        ('cat', cat_encoder, categorical_cols)
    ]
)

# 7. Full pipeline with scaling after combining
mlp_input_pipeline = Pipeline(steps=[
    ('column_processing', column_pipeline),
    ('scaling', StandardScaler())
])

# 8. Fit and transform
X_train_mlp = mlp_input_pipeline.fit_transform(train_data)
X_val_mlp = mlp_input_pipeline.transform(val_data)

C:\Users\Harshada\AppData\Local\Temp\ipykernel_26512\2503547276.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop(['value', 'year'], axis=1, inplace=True)
C:\Users\Harshada\AppData\Local\Temp\ipykernel_26512\2503547276.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data.drop(['value', 'year'], axis=1, inplace=True)


In [ ]:
# 1. Reshape target arrays for scaling
train_target_reshaped = train_target.reshape(-1, 1)
val_target_reshaped = val_target.reshape(-1, 1)

# 2. Initialize and apply StandardScaler
target_scaler = StandardScaler()
train_target_scaled = target_scaler.fit_transform(train_target_reshaped)
val_target_scaled = target_scaler.transform(val_target_reshaped)

In [ ]:
class YieldDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # Regression output shape

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
class YieldMLP(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(YieldMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_layers[0]),
            nn.ReLU(),
            nn.Linear(hidden_layers[0], hidden_layers[1]),
            nn.ReLU(),
            nn.Linear(hidden_layers[1], 1),
            nn.ReLU()  # Optional: use only if predicting scaled/normalized positive yield
        )

    def forward(self, x):
        return self.model(x)


from torch.utils.data import DataLoader

batch_size = 32

# 1. Create dataset objects
train_dataset = YieldDataset(X_train_mlp, train_target_scaled)
val_dataset = YieldDataset(X_val_mlp, val_target_scaled)

# 2. Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)

In [ ]:
input_size = X_train_mlp.shape[1]
hidden_units = [64, 32]
learning_rate = 0.0001
num_epochs = 10

mlp_model = YieldMLP(input_dim=input_size, hidden_layers=hidden_units)
optimizer = optim.Adam(mlp_model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()


best_val_loss = float('inf')
epoch_losses = []

for epoch in range(num_epochs):
    mlp_model.train()
    epoch_train_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        predictions = mlp_model(X_batch)
        loss = loss_fn(predictions, y_batch)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()

    # Validation
    mlp_model.eval()
    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            val_predictions = mlp_model(X_val_batch)
            val_loss = loss_fn(val_predictions, y_val_batch)

    # Record losses
    avg_train_loss = epoch_train_loss / len(train_loader)
    epoch_losses.append((avg_train_loss, val_loss.item()))

    # Early stopping logic
    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        best_model_weights = mlp_model.state_dict()
        best_epoch = epoch
        print(f"Epoch {epoch}: Improved validation loss to {val_loss.item():.4f}")

    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss.item():.4f}")

Epoch 0: Improved validation loss to 0.4815
Epoch 0 | Train Loss: 0.6631 | Val Loss: 0.4815
Epoch 1: Improved validation loss to 0.4247
Epoch 2: Improved validation loss to 0.3765
Epoch 3: Improved validation loss to 0.3400
Epoch 4: Improved validation loss to 0.3125
Epoch 5: Improved validation loss to 0.2948
Epoch 6: Improved validation loss to 0.2863
Epoch 7: Improved validation loss to 0.2740
Epoch 8: Improved validation loss to 0.2646
Epoch 9: Improved validation loss to 0.2636


In [ ]:
# Load best model weights
mlp_model.load_state_dict(best_model_weights)
mlp_model.eval()

# Evaluate on validation set
with torch.no_grad():
    final_val_preds = mlp_model(torch.tensor(X_val_mlp, dtype=torch.float32))
    final_mse = loss_fn(final_val_preds, torch.tensor(val_target_scaled, dtype=torch.float32).view(-1, 1))
    print(f"\nFinal Validation MSE: {final_mse.item():.4f}")


Final Validation MSE: 0.2636


In [ ]:

# Predict using best model
mlp_model.eval()
with torch.no_grad():
    val_preds_scaled = mlp_model(torch.tensor(X_val_mlp, dtype=torch.float32)).numpy()
    val_true_scaled = val_target_scaled  # already a NumPy array

# Inverse transform the scaled predictions and true values
val_preds_unscaled = target_scaler.inverse_transform(val_preds_scaled)
val_true_unscaled = target_scaler.inverse_transform(val_true_scaled)

# Compute evaluation metrics
r2 = r2_score(val_true_unscaled, val_preds_unscaled)
rmse = np.sqrt(mean_squared_error(val_true_unscaled, val_preds_unscaled))
mae = mean_absolute_error(val_true_unscaled, val_preds_unscaled)

# Print results
print(f"Validation R² Score: {r2:.4f}")
print(f"Validation RMSE: {rmse:.2f}")
print(f"Validation MAE: {mae:.2f}")

Validation R² Score: 0.7596
Validation RMSE: 7441.73
Validation MAE: 5715.68


In [ ]:
# Flatten predictions and ground truth arrays
true_values = val_true_unscaled.flatten()
predicted_values = val_preds_unscaled.flatten()

# Construct DataFrame
validation_results_df = pd.DataFrame({
    'True_Yield': true_values,
    'Predicted_Yield': predicted_values
})

# Preview the results
print(validation_results_df.head())

   True_Yield  Predicted_Yield
0     10600.0      9369.880859
1      7527.9      9369.880859
2      1375.0      9369.880859
3     10663.5      9369.880859
4      4150.5      9369.880859


In [ ]:
validation_results_df.head()

,True_Yield,Predicted_Yield
0,10600.0,9369.880859
1,7527.9,9369.880859
2,1375.0,9369.880859
3,10663.5,9369.880859
4,4150.5,9369.880859


In [ ]:
path = r'C:\Users\Harshada\Downloads\OneDrive_2025-05-07'